# CaliDHRI Day 2: Afternoon

## Topic Modeling with Gensim

Topic modeling is a popular approach to "distant reading" a corpus. It is a kind of unsupervised machine learning. Unlike in supervised methods where the human provides labeled training data from which the machine learns, in unsupervised machine learning, the machine learns from unlabeled data. There are benefits and drawbacks to both approaches. Topic modeling is a useful tool for understanding the subject matter of a dataset -- sometimes one that is too large for close reading or as a supplement to close reading.

We will be doing LDA topic modeling. LDA stands for latent Dirichlet allocation. Don't get hung up on the terminology! 

### Topics are probabilities of aboutness

Here's how the basic logic of LDA topic modeling works:
* Start with a corpus of documents with any pre-processing/cleaning done.
* Represent each document as a bag-of-words. In the BOW model, word order is lost and it just matters that the word occured within the bag (document).
* The BOWs are represented as a term-document matrix consisting of columns and rows for each word's occurance.
* The term-document matrices are fed to the algorithm. Each word in each document is compared (Is Word1 in Bag1? Yes. Is Word1 in Bag2? Yes. Is Word1 in Bag3? No. Is Word2 in Bag1? No. Is Word2 in Bag2? Yes. Is Word2 in Bag3? No. And so on.)
* The computer learns from this process which words are likely to occur within the same document. The logic of LDA is that words that co-occur within a document are likely about the same thing.
* The algorithm generates topics, which are probablistic groupings of words that are likely to co-occur within the corpus. 
* The algorithm does not tell you want the topics are about and it does not name the topics. That's up to the human!

## Set up

In this notebook, we'll be using several libraries to prepare text from The Liberator for the algorithm, create topic models, and visualize the topics:
* NLTK: data preparation
* Gensim: topic modeling
* Altair: visualization
* ipywidgets: visualization
* Matplotlib: visualization


These are already installed on JupyterHub, but if you were working locally, you'd want to make sure you installed them before trying to use them in your notebook. 

## The data 

Here are the modules we'll use to load our data into the notebook:
* Glob

The import statements below will make them available for us to use in our notebook.

In [ ]:
import glob

Now we need to direct the notebook to the 187 files of the Liberator we'll be topic modeling. Each file is an issue of the Liberator. We haven't done any cleaning of the OCR for these files. 

Learn more about the Liberator in [these slides](https://docs.google.com/presentation/d/1gwAISOnn3Evq_LkX_0vhsYy0iTv7s5GHLUEGVKzN-tc/edit?usp=sharing). 

First we create a variable called directory_path, and we set that to the directory name where our files are stored. Then we can use glob, a handy module to navigate a directory structure, to set all of the files under directory_path to another variable called text files.

In [ ]:
directory_path = 'libSeparate'
text_files = glob.glob(f"{directory_path}/*")

Let's check to see that the files were brought in. We can use the list index to call the first text file. 

In [ ]:
# change the number in the brackets to see a different file
text_files[0]

The next step is to take those text files and make them into a Python list. First we create an empty list, listOfLib, and using a for loop, we iterate through the files in text_files, opening and reading them, and then apending them to listOfLib.

In [ ]:
listOfLib = []
for i in text_files:
    try:
        with open(i) as f:
            listOfLib.append(f.read())
    except:
        pass

Let's check to make sure we got all the files. We can use the notation len() to see the length of both text_files and listOfLib are. The length is the number of items in the list. 

In [ ]:
len(text_files)

In [ ]:
len(listOfLib)

With the data now stored in our listOfLib, we can also call each of the items in the list using the notation below. The first brackets refer to a specific item in our list, in this case the 10th item. (Python starts counting at 0.) The second brackets are optional, and they are restricting what is returned to a slice of the data--characters 0 to 300. If we didn't slice, then we'd return the text of one entire issue of the Liberator. 

In [ ]:
listOfLib[9][0:300]

## Prepare the data

Once the data is loaded and in our listOfLib, then we can move to preparing it for analysis. We are going to remove the stopwords, tokenize, and lemmatize the text. For this we will utilize NLTK's built-in stopwords, tokenizer, and lemmatizer. 

We import the NLTK modules below, plus a module called punctuation that will allow us to remove punctuation from the text. 

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

Let's check out the NLTK stopword list. 

In [ ]:
standardStop = stopwords.words('english')
print (standardStop)

The standard stopwords may not be sufficient for the text we will analysis from the Liberator. We can add stopwords by creating a list called extraStop that we will combine with the standard list in a new list. The new list will be called myStopWords, and it combines the punctuation marks, standardStop, and extraStop lists. 

In [ ]:
extraStop = ['``',"''","'re","'s","'re",'``',"''","'ll","--","\'\'","...", "n\'t",'one','would','use','subject','from',
             "\'m","\'ve", "los", "angeles", "liberator", "phone", "street", "mrs."]

In [ ]:
myStopWords = list(punctuation) + standardStop + extraStop

In [ ]:
print(myStopWords)

Now we'll create a function to tokenize and lemmatize the text. As you'll remember from this morning, lemmatization groups together forms of a word.  

Let's take note of all the preprocessing happening in this function:

* We create 2 empty lists called listOfLibWords and allLibWords
* Then, in a for loop, we go through each of the Liberators stored in our listOfLib and...
    * For ever word, we lowercase and tokenize it if it is not on myStopWords and it has more than 3 letters
    * The tokens are stored in a temporary list called processedText
    * We append the processedText list to the listOfLibWords
* And in another for loop, we go through each token in processedText and add it to the allLibWords list

In [ ]:
listOfLibWords = []
allLibWords = []
for i in listOfLib:
    processedText = [w for w in word_tokenize(i.lower()) if w not in myStopWords and len(w) > 3]
    listOfLibWords.append(processedText)
    for token in processedText:
        allLibWords.append(token)

Then we need to lemmatize the tokens in allLibWords. We'll create yet another empty list called listOfLemWords. We'll name a variable wordnet_lemmatizer and set it to NLTK's WordNetLemmatizer. Then in another for loop, we'll iterate through the tokens in listOfLibWords and add them to listOfLemWords once they have been lemmatized.

In [ ]:
listOfLemWords = []
wordnet_lemmatizer = WordNetLemmatizer()
for i in listOfLibWords:
    listOfLemWords.append([wordnet_lemmatizer.lemmatize(w) for w in i])

Let's take a look at a slice of tokens from list item 6.

In [ ]:
listOfLemWords[6][0:25]

## Create the bag-of-words

Gensim expects the data in a certain format in order to do topic modeling. First, we need to create a Python dictionary from our list of words. Then we need to translate the dictionary into a bag-of-words. 

But before any of that, we need to import Gensim and the corpora and models modules.

In [ ]:
import gensim
from gensim import corpora

Now we can define a variable called dictionary. Gensim's corpora module will map the words in each document to integer IDs, in other words, assigning each unique word in the corpus a corresponding number.

In [ ]:
dictionary = corpora.Dictionary(listOfLemWords) 

We can check the ID of any token an our corpus. 

In [ ]:
#try looking up another word to find its ID
print(dictionary.token2id['broadway'])

Then, we define a new variable called corpus. Gensim has a module called doc2bow that will convert a document into the bag-of-words format. The BOW will be a list of tuples -- consisting of word ID and frequency pairs -- that represent each document in our corpus. 

In [ ]:
corpus = [dictionary.doc2bow(text) for text in listOfLemWords]

In [ ]:
print(corpus[9][0:300])

## Run the model and view topics

Gensim has a module called models for LDA topic modeling. We'll import the models module and then create our topics. 

In [ ]:
from gensim import models

There are parameters that we have the option to set when creating the topic model. We're using just couple here:

* num_topics: the number of topics our model will create. This is a parameter you can adjust based on your corpus size and the length of the documents. We are starting with 10 topics.
* passes: the number of times the algorithm will pass through the documents. A higher number is likely to yield higher quality topics, but will take more time to run. 

You can find the full list of available parameters on this webpage: https://radimrehurek.com/gensim/models/word2vec.html. 

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics=10, 
                                           id2word = dictionary, 
                                           passes=20)

The notation show_topics() will return the 10 most salient words in each of our topics. 

In [ ]:
lda_model.show_topics()
#try adding a topic number (e.g., 5) and topn= 20 to see twenty words in topic 5, for example show_topics(9, topn=20)

The code below will print a cleaner view of the top 20 words in our topics.

In [ ]:
for i in range(10):
    print('Topic '+str(i))
    for j in lda_model.show_topic(i, topn=20):
            print(j[0])
    print('\n')

## Visualize topics

Visualizing our topics will help us make sense of them. The code blocks below are a bit more complicated than what we saw above. 

First, let's import the modules we'll use in this section. 

* Altair: visualization library for Python
* ipywidgets: will make our visualizations interactive
* Pandas: common Python library for data science that is useful when working with tabular data
* Matplotlib: a very popular Python visualization library
* Seaborn: another Python visualization library 

Where you see the statements written "import ______ as ____", that's a way to abbreviate a module's name, mostly for the convenience of not typing the full name later. The abbreviations used here are common, so you may seem them if you look at other code in the future. 

In [ ]:
import altair as alt
import ipywidgets
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pyLDAvis is another frequently used Python library for visualizing the outputs of Gensim topic models. The code is a bit simpler than what we're using here, but we think these custom visualizations are nicer! If you wanted to use pyLDAvis, you'd need to run:

In [ ]:
#import pyLDAvis
#import pyLDAvis.gensim_models
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, id2word, mds="mmds", R=30)
#vis

Let's get back to our visualiations. First we want to create a list of just the tokens in the topics. 

Here's what's happening below:
* Create a variable called topics and set it to show_topics
* Create an empty list called topic_words
* Iterate through topics and append the tokens to the topic_words list

At the end, we print topic_words

In [ ]:
topics=lda_model.show_topics(5, 10,formatted=False)
topic_words = []
for topic in topics:
    topic_words.append([wd[0] for wd in topic[1]])
print(topic_words)

Then, we want to create a Pandas dataframe from our corpus. Try not to worry to much about the syntax below, which mixes in Pandas syntax that we haven't yet seen. The important thing to know is that we are creating a dataframe, which you can think about like superpowered, code-accessible spreadsheet. Our dataframe will include the probability for each topic in each file in our corpus. 

In [ ]:
ntopics = 10
dftopics = pd.DataFrame({'corpus':corpus})
filenames = []
topiclist = []
topicprob = []
topiccorpus = []
for i,row in dftopics.iterrows():
    probs = [0]*ntopics
    for t in lda_model.get_document_topics(row['corpus']):
        probs[t[0]] = t[1]
    for j in range(ntopics):
        topiccorpus.append('file'+str(i))
        topiclist.append(j)
        topicprob.append(probs[j])
topic_dataframe = pd.DataFrame({'corpus':topiccorpus,'topic':topiclist,'prob':topicprob})

If we call the dataframe, then we can get a feel for the tabular format.

In [ ]:
topic_dataframe

Next we can sort the dataframe so that it goes in ascending order by topic and probability. 

In [ ]:
topic_dataframe = topic_dataframe.sort_values(by=['topic','prob'],ascending=[True,False])

The following code is a lot to digest, but it will take our dataframe and generate an interactive visualization showing the dispersion of topics within our issues of the Liberator. 

In [ ]:
ntopics = 10
numwords = 10
selection = alt.selection_single(on='mouseover',fields=['topic'], bind='legend')
selopac = alt.selection_single(on='mouseover',fields=['topic'],bind='legend')
def topicbars(Topic):
    a = Topic
    topics_df_sorted = topic_dataframe[topic_dataframe['topic']==a].sort_values(by=['topic','prob'],ascending=False)
    topics_df_sorted = topics_df_sorted.append(topic_dataframe[topic_dataframe['topic']!=a])

    x=lda_model.show_topics(num_topics=ntopics, num_words=numwords,formatted=False)
    topic_words = []
    for topic in x:
        topic_words.append([wd[0] for wd in topic[1]])
    topics_df_sorted['topicwords'] = ''
    for i,row in topics_df_sorted.iterrows():
        topics_df_sorted.loc[i,'topicwords'] = ', '.join(topic_words[row['topic']])
    print("\n"+', '.join(topic_words[Topic])+"\n")
    
    chart = alt.Chart(topics_df_sorted).mark_bar().encode(
        x = 'sum(prob)',
        y = alt.X('corpus',sort=None),
        color = alt.Color('topic:N'),
        order = alt.Order('corpus'),
        opacity=alt.condition(selopac, alt.value(1), alt.value(0.5)),
        tooltip=['corpus', 'topic', 'prob', 'topicwords']
    ).add_selection(selopac)
        
    return chart

ipywidgets.interact(topicbars,Topic=range(ntopics))

Then we can modify our dataframe so that topic is now rendered as a column. This will set us up for another visualization that will show how prevalent each topic is in each issue of the Liberator. 

In [ ]:
topic_dataframe.pivot(index='corpus',columns='topic',values='prob')

Let's integrate that code above into the code below to create a new dataframe where the words in each topic will now be brought in, instead of the topic number. Then we'll call the new dataframe. 

In [ ]:
ntopics = 10
numwords = 10

df_topic_words = topic_dataframe.pivot(index='corpus',columns='topic',values='prob')

x=lda_model.show_topics(num_topics=ntopics, num_words=numwords,formatted=False)
topic_words = []
for topic in x:
    topic_words.append(','.join([wd[0] for wd in topic[1]]))
df_topic_words.columns = topic_words

In [ ]:
df_topic_words

In [ ]:
f, ax = plt.subplots(figsize=(20, 80))
ax = sns.heatmap(df_topic_words,
                 cmap="YlGnBu",
                 annot=True)
plt.tick_params(bottom=False, labelbottom=False, top=True, labeltop=True)
plt.xticks(rotation=30, ha='left')
plt.yticks(rotation=0)
plt.show()